In [ ]:
import yaml
import os
from a_star import AStarTime, AStarSearch
import numpy as np
from tqdm import tqdm
import time


In [ ]:
import scipy.io as scio
def save_mat(path, to_save):
    '''
    Save data to mat file
    Args:
        path: path to save mat file
        to_save: data to save in the mat
    '''
    scio.savemat(path, to_save) 

# Global Replanning Baseline Codes
Version: 1.0

Author: Weizhe Lin @ Feb 2021

In [ ]:

# next_loc, agent_path, num_replan = follow_path(env_img, agent_path, end, num_replan)

def follow_path(img, agent_path, end, num_replan):
    '''
    AStar replanning function. Input one agent's plan and ask the agent to move one step. 
    If the next step is prohibited, replanning is executed to obtain new path using AStar, taking other
    agents as obstacles.
    Args:
        img: map
        agent_path: current agent path
        end: goal position
        num_replan: number of replanning, to be accumulated during execution
    Returns:
        (loc_x, loc_y): new locations of the agent
        agent_path: planned agent path, might have been replanned. Otherwise follow the original plan
        num_replan: accumulated number of replanning, +1 if replanning is executed
    '''
    old_path = agent_path.copy()
    loc_x = np.where(img==2)[0][0]
    loc_y = np.where(img==2)[1][0]
    if len(old_path) == 0:
        # no path to be computed
        next_x, next_y = loc_x, loc_y
#     end_x, end_y = agent_path[-1][0], agent_path[-1][1]
#     end = (end_x, end_y)
    else:
        next_x, next_y = agent_path[0][0], agent_path[0][1]
    if img[next_x, next_y] == -1 or img[next_x, next_y] == 0:
        if (next_x, next_y) == end:
            print('reach target')
            return (next_x, next_y), agent_path, num_replan
        else: 
            return (next_x, next_y), agent_path[1:], num_replan
    else:
        agent_path = AStarSearch(img, (loc_x, loc_y), end)
        num_replan += 1
        if len(agent_path) == 0:
#             print('wait')
            return (loc_x, loc_y), old_path, num_replan
#         print('num_replan:', num_replan)
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                if img[i,j] == -1:
                    img[i,j] = 0
#         img = plot_path(img, agent_path)
        return (loc_x, loc_y), agent_path[1:], num_replan

In [ ]:
DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map20x20_density_p1/10_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000

In [ ]:
data = []
ecbs_data = []
for root, dirs, files in os.walk(os.path.join(DATA_FOLDER, 'input')):
    for f in files:
        print(len(data), end='\r')
        if len(data) >= DATA_NUM:
            break
        IDMAP = f.split('IDMap')[1].split('_')[0]
        IDCase = f.split('IDCase')[1].split('.yaml')[0]
        IDMAP = int(IDMAP)
        IDCase = int(IDCase)
        if IDMAP >= DATA_RANGE[0] and IDMAP <= DATA_RANGE[1]:
            with open(os.path.join(root, f),'r',encoding='utf-8') as fs:
                cont = fs.read()
                x = yaml.load(cont)
                
#             print(IDMAP, IDCase)
            
            output_name = f.replace('input', 'output').split('.yaml')[0] + '_ECBS.yaml'
            output_path = os.path.join(DATA_FOLDER, 'output_ECBS', output_name)
            if os.path.exists(output_path):
                with open(output_path ,'r',encoding='utf-8') as fs:
                    cont = fs.read()
                    y = yaml.load(cont)
#                     print(output_name)
                data.append(x)
                ecbs_data.append(y)

print(len(data))
print(len(ecbs_data))

In [ ]:
# flowtime_increase_list = []
# time_cost_list = []
# all_success_count = []
# individual_success_count = []
# dim_num = None

# for input_data, output_data in tqdm(zip(data, ecbs_data), total=len(data)):
#     success_count = []
#     start_time = time.time()
#     expert_makespan = output_data['statistics']['makespan']
#     expert_cost = output_data['statistics']['cost']
    
# #     print('===')
# #     print(input_data)
# #     print(output_data)
#     env = np.zeros(input_data['map']['dimensions'])
#     for obs in input_data['map']['obstacles']:
#         env[obs[0], obs[1]] = 1
#     if not dim_num:
#         dim_num = [input_data['map']['dimensions'][0], len(input_data['agents'])]
#     all_start = []
#     all_end = []
    
#     agent_paths = []
#     agent_positions = []
#     for agent_data in input_data['agents']:
#         start = agent_data['start']
#         end = agent_data['goal']
#         all_start.append(start)
#         all_end.append(end)
#         agent_positions.append(start)
    
#     # init
    
#     for i in range(len(input_data['agents'])):
#         img = env.copy()
# #         print('agent', i, (agent_positions[i][0], agent_positions[i][1]), all_end[i])
#         for other_agent_pos in [agent_pos for agent_index, 
#                                 agent_pos in enumerate(agent_positions) if agent_index!=i]:
#             img[other_agent_pos[0], other_agent_pos[1]] = 1
# #             print('added as obs', other_agent_pos)
#         agent_path = AStarSearch(img, (agent_positions[i][0], agent_positions[i][1]), all_end[i])
#         agent_paths.append(agent_path[1:])
    
#     num_replan = [0] * len(input_data['agents'])
#     flag_done = [0] * len(input_data['agents'])
#     current_step=1
#     while current_step < expert_makespan*3:
# #         print('======roound', current_step)
#         # start moving
#         for i in range(len(input_data['agents'])):
#             if flag_done[i] != 0:
# #                 print('ignore agent', i, agent_positions[i], 'num_replan', num_replan[i])
#                 continue
#             img = env.copy()
#             for other_agent_pos in [agent_pos for agent_index, 
#                                     agent_pos in enumerate(agent_positions) if agent_index!=i]:
#                 img[other_agent_pos[0], other_agent_pos[1]] = 1
#             img[agent_positions[i][0], agent_positions[i][1]] = 2
# #             img[all_end[i][0], all_end[i][1]] = -1
# #             print('old agent path', agent_paths[i])
#             next_loc, agent_path, num_replan[i] = follow_path(img, agent_paths[i], all_end[i], num_replan[i])
# #             print('next_loc', next_loc, 'num_replan', num_replan[i])
# #             print('now agent path', agent_path)
#             agent_positions[i] = next_loc
#             agent_paths[i] = agent_path
#             if agent_positions[i][0] == all_end[i][0] and agent_positions[i][1] == all_end[i][1]:
# #                 print('agent', i, 'reach goal')
#                 flag_done[i] = current_step
#         if 0 not in flag_done:
# #             print('finished. cost:', current_step)
#             break
#         current_step+=1
    
#     success_count = []
#     cost = 0
#     makespan = current_step
#     for agent_flag in flag_done:
#         if agent_flag == 0:
#             success_count.append(0)
#             cost += expert_makespan*3
#         else:
#             cost += agent_flag
#             success_count.append(1)
#     print(flag_done)
#     end_time = time.time()
#     time_elapsed = end_time - start_time
#     time_cost_list.append(time_elapsed)
#     flowtime_increase = cost/expert_cost-1
#     flowtime_increase_list.append(flowtime_increase)
#     individual_success_count += success_count
# #     print(time_elapsed, flowtime_increase)
# #     print((0 not in success_count), success_count)
# #     if (0 in success_count):
# #         print(input_data)
# #         print(output_data)
# #         print(all_paths[success_count.index(0)])
#     all_success_count.append((0 not in success_count))

In [ ]:
# print('{}x{}({})'.format(dim_num[0], dim_num[0], dim_num[1]))
# flowtime_increase_array = np.array(flowtime_increase_list)
# time_cost_array = np.array(time_cost_list)
# all_success_array = np.array(all_success_count)
# individual_success_array = np.array(individual_success_count)
# print('FT_increase;{};{}'.format(np.mean(flowtime_increase_array), np.std(flowtime_increase_array)))
# print('time_cost;{};{}'.format(np.mean(time_cost_array), np.std(time_cost_array)))
# print('all_success;{};{}'.format(np.mean(all_success_array), np.std(all_success_array)))
# print('individual_success_rate;{};{}'.format(np.mean(individual_success_array), np.std(individual_success_array)))
# print('{}x{}({})'.format(dim_num[0], dim_num[0], dim_num[1]),';','{};{};{};{};{};{};{};{}'.format(
#         np.mean(flowtime_increase_array), np.std(flowtime_increase_array),
#         np.mean(time_cost_array), np.std(time_cost_array),
#         np.mean(all_success_array), np.std(all_success_array),
#         np.mean(individual_success_array), np.std(individual_success_array),
#     ))

In [ ]:
LOG_TIME = int(time.time())
DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map20x20_density_p1/10_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 4500
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map28x28_density_p1/20_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map35x35_density_p1/30_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map40x40_density_p1/40_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map45x45_density_p1/50_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map50x50_density_p1/60_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map65x65_density_p1/100_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

In [ ]:
LOG_TIME = int(time.time())
DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map20x20_density_p1/10_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 4500
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map28x28_density_p1/20_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map35x35_density_p1/30_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map40x40_density_p1/40_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map45x45_density_p1/50_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map50x50_density_p1/60_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map65x65_density_p1/100_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

In [ ]:
LOG_TIME = 1612443878
DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/10_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/20_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/30_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/40_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/50_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/100_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

In [ ]:
DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map65x65_density_p1/100_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 5
run_in_pipeline()

In [ ]:
def run_in_pipeline():
    data = []
    ecbs_data = []
    for root, dirs, files in os.walk(os.path.join(DATA_FOLDER, 'input')):
        for f in files:
            print('loading...', len(data), end='\r')
            if len(data) >= DATA_NUM:
                break
            IDMAP = f.split('IDMap')[1].split('_')[0]
            IDCase = f.split('IDCase')[1].split('.yaml')[0]
            IDMAP = int(IDMAP)
            IDCase = int(IDCase)
            if IDMAP >= DATA_RANGE[0] and IDMAP <= DATA_RANGE[1]:
                with open(os.path.join(root, f),'r',encoding='utf-8') as fs:
                    cont = fs.read()
                    x = yaml.load(cont)

                output_name = f.replace('input', 'output').split('.yaml')[0] + '_ECBS.yaml'
                output_path = os.path.join(DATA_FOLDER, 'output_ECBS', output_name)
                if os.path.exists(output_path):
                    with open(output_path ,'r',encoding='utf-8') as fs:
                        cont = fs.read()
                        y = yaml.load(cont)
                    data.append(x)
                    ecbs_data.append(y)
          
    print('finished loading:', len(data))
    print(len(ecbs_data))

    flowtime_increase_list = []
    makespan_list = []
    time_cost_list = []
    all_success_count = []
    individual_success_count = []
    num_reachGoal_list = []
    dim_num = None

    for input_data, output_data in tqdm(zip(data, ecbs_data), total=len(data)):
        success_count = []
        start_time = time.time()
        expert_makespan = output_data['statistics']['makespan']
        expert_cost = output_data['statistics']['cost']

#         print('===')
#         print(input_data)
#         print(output_data)
        env = np.zeros(input_data['map']['dimensions'])
        for obs in input_data['map']['obstacles']:
            env[obs[0], obs[1]] = 1
        if not dim_num:
            dim_num = [input_data['map']['dimensions'][0], len(input_data['agents'])]
        all_start = []
        all_end = []

        agent_paths = []
        agent_positions = []
        for agent_data in input_data['agents']:
            start = agent_data['start']
            end = agent_data['goal']
            all_start.append(start)
            all_end.append(end)
            agent_positions.append(start)

        # init

        for i in range(len(input_data['agents'])):
            img = env.copy()
    #         print('agent', i, (agent_positions[i][0], agent_positions[i][1]), all_end[i])
            for other_agent_pos in [agent_pos for agent_index, 
                                    agent_pos in enumerate(agent_positions) if agent_index!=i]:
                img[other_agent_pos[0], other_agent_pos[1]] = 1
    #             print('added as obs', other_agent_pos)
            agent_path = AStarSearch(img, (agent_positions[i][0], agent_positions[i][1]), all_end[i])
            agent_paths.append(agent_path[1:])

        num_replan = [0] * len(input_data['agents'])
        flag_done = [0] * len(input_data['agents'])
        current_step=1
        while current_step < expert_makespan*3:
#             print('======roound', current_step)
            # start moving
            for i in range(len(input_data['agents'])):
                if flag_done[i] != 0:
#                     print('ignore agent', i, agent_positions[i], 'num_replan', num_replan[i])
                    continue
                img = env.copy()
                # fill in other agents as obstacles
                for other_agent_pos in [agent_pos for agent_index, 
                                        agent_pos in enumerate(agent_positions) if agent_index!=i]:
                    img[other_agent_pos[0], other_agent_pos[1]] = 1
                img[agent_positions[i][0], agent_positions[i][1]] = 2
#                 img[all_end[i][0], all_end[i][1]] = -1
#                 print('old agent path', agent_paths[i])
                next_loc, agent_path, num_replan[i] = follow_path(img, agent_paths[i], all_end[i], num_replan[i])
#                 print('next_loc', next_loc, 'num_replan', num_replan[i])
#                 print('now agent path', agent_path)
                agent_positions[i] = next_loc
                agent_paths[i] = agent_path
                if agent_positions[i][0] == all_end[i][0] and agent_positions[i][1] == all_end[i][1]:
#                     print('agent', i, 'reach goal')
                    flag_done[i] = current_step
            if 0 not in flag_done:
#                 print('finished. cost:', current_step)
                break
            current_step+=1

        success_count = []
        cost = 0
        makespan = current_step
        makespan_list.append(makespan)
        print(flag_done)
        print('expert makespan', expert_makespan)
        print('expert cost', expert_cost)
        print('total in flag done', sum(flag_done))
        for agent_flag in flag_done:
            if agent_flag == 0:
                success_count.append(0)
                cost += expert_makespan*3
            else:
                cost += agent_flag
                success_count.append(1)
        print('computed cost', cost)
        end_time = time.time()
        time_elapsed = end_time - start_time
        time_cost_list.append(time_elapsed)
        flowtime_increase = cost/expert_cost-1
        print('FT increase', flowtime_increase)
        flowtime_increase_list.append(flowtime_increase)
        individual_success_count += success_count
        all_success_count.append((0 not in success_count))
        
        num_reachGoal_list.append(np.count_nonzero(np.array(success_count)))
        
    print('{}x{}({})'.format(dim_num[0], dim_num[0], dim_num[1]))
    flowtime_increase_array = np.array(flowtime_increase_list)
    makespan_array = np.array(makespan_list)
    time_cost_array = np.array(time_cost_list)
    all_success_array = np.array(all_success_count)
    individual_success_array = np.array(individual_success_count)
    print('FT_increase;{};{}'.format(np.mean(flowtime_increase_array), np.std(flowtime_increase_array)))
    print('time_cost;{};{}'.format(np.mean(time_cost_array), np.std(time_cost_array)))
    print('all_success;{};{}'.format(np.mean(all_success_array), np.std(all_success_array)))
    print('individual_success_rate;{};{}'.format(np.mean(individual_success_array), np.std(individual_success_array)))
    
    log_time = LOG_TIME
    mat_data = {
        'rate_ReachGoal':[[np.mean(all_success_array)]],
        'num_agents_trained':[[dim_num[1]]],
        'num_agents_testing':[[dim_num[1]]],
        'map_size_testing':[[dim_num[0], dim_num[0]]],
        'map_size_trained': [[dim_num[0], dim_num[0]]],
        'map_density_trained': [[0.1]],
        'map_density_testing': [[0.1]],
        'K':[[0]],
        'trained_model_epoch':[[0]],
        'log_time':[[log_time]],
        'std_deltaMP': [[np.std(makespan_array)]],
        'mean_deltaMP':[[np.mean(makespan_array)]],
        'list_deltaMP':[[makespan_list]],
        'mean_deltaFT':[[np.mean(flowtime_increase_array)]],
        'std_deltaFT':[[np.std(flowtime_increase_array)]],
        'list_deltaFT': [flowtime_increase_list],
        'list_reachGoal':all_success_count,
        'list_computationTime':[time_cost_list],
        'list_numAgentReachGoal':[num_reachGoal_list],
        'action_policy': 'exp_multinorm',
        'hidden_state': [[0]],
    }
    pprint(mat_data)
    file_name = 'REPLAN_{}x{}({})_{}_exp_multinorm.mat'.format(dim_num[0], dim_num[0], dim_num[1], log_time)
    save_mat(file_name, mat_data)
    print('{}x{}({})'.format(dim_num[0], dim_num[0], dim_num[1]),';','{};{};{};{};{};{};{};{}'.format(
        np.mean(flowtime_increase_array), np.std(flowtime_increase_array),
        np.mean(time_cost_array), np.std(time_cost_array),
        np.mean(all_success_array), np.std(all_success_array),
        np.mean(individual_success_array), np.std(individual_success_array),
    ))
    
    print('========done========')

In [ ]:
rate_ReachGoal = mat_data['rate_ReachGoal'][0][0]
mean_deltaFT = mat_data['mean_deltaFT'][0][0]
mean_deltaMP = mat_data['mean_deltaMP'][0][0]
hidden_state = mat_data['hidden_state'][0][0]
trained_model_epoch = mat_data.get('trained_model_epoch', [[-1]])
trained_model_epoch = trained_model_epoch[0][0]
num_agents_trained = mat_data['num_agents_trained'][0][0]
num_agents_testing = mat_data['num_agents_testing'][0][0]
map_size_testing = mat_data['map_size_testing'][0][0]
K = mat_data['K'][0][0]
cleaned_data = {
    'filename': file,
    'type': data_type,
    'action_policy': action_policy,
    'map_size_trained': mat_data['map_size_trained'][0],
    'map_density_trained': mat_data['map_density_trained'][0][0],
    'num_agents_trained': mat_data['num_agents_trained'][0][0],
    'map_size_testing': mat_data['map_size_testing'][0],
    'map_size_testing_int': mat_data['map_size_testing'][0][0],
    'map_density_testing': mat_data['map_density_testing'][0][0],
    'num_agents_testing': mat_data['num_agents_testing'][0][0],
    'log_time': log_time,
    'K': K,
    # 'data_set' : mat_data['data_set'][0][0],
    'hidden_state': hidden_state,
    'rate_ReachGoal': rate_ReachGoal,
    'list_reachGoal': mat_data.get('list_reachGoal', None),
    'list_computationTime': mat_data.get('list_computationTime', None),
    'mean_deltaFT': mean_deltaFT,
    'std_deltaMP': mat_data['std_deltaMP'][0][0],
    'mean_deltaMP': mean_deltaMP,
    'std_deltaFT': mat_data['std_deltaFT'][0][0],
    'trained_model_epoch': trained_model_epoch,
    'list_deltaFT': mat_data['list_deltaFT'][0],
    # 'list_FT_predict': mat_data['list_FT_predict'][0],
    # 'list_FT_target': mat_data['list_FT_target'][0],
    #
    # 'list_reachGoal': mat_data['list_reachGoal'][0],
    'list_numAgentReachGoal': mat_data['list_numAgentReachGoal'][0],
    'hist_numAgentReachGoal': mat_data['hist_numAgentReachGoal'][0],
}